In [ ]:
import glob
import os
import pathlib

import numpy as np
import pandas as pd

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import functional as F

from torchvision import transforms
from torchinfo import torchinfo
from tqdm import tqdm
import matplotlib.pyplot as plt

import albumentations as A
import torch.nn.functional as F

from PIL import Image

import torchmetrics
from torchvision.utils import save_image, make_grid
import cv2
import os, glob

from utils.networks import CNN, EfficientUNet_B0


device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
FACERECOG_PATH = 'Models/mask_model.pth'
INFILLER_PATH  = 'Models/efficientnet_tuned.pth'


def load_file(path, device = device):
    with open(path, 'rb') as f:
        return torch.load(f, map_location = device)

In [ ]:
# loading the mask-detection model

classifier = CNN().to(device)
classifier.load_state_dict(load_file(FACERECOG_PATH))

In [ ]:
# loading the inpainting model

infiller = EfficientUNet_B0().to(device)
infiller.load_state_dict(load_file(INFILLER_PATH))

In [ ]:
classifier.eval()
infiller.eval()

classifier.requires_grad_(False)
infiller.requires_grad_(False)

print("")

In [ ]:
resize_transformation = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5),
    transforms.Resize((64, 64), antialias = True)
])

def masked_inference_pipeline(image):
    with torch.no_grad():
        img = resize_transformation(image)[None, ...]
        img = img.to(device)
        return classifier(img).item() <= 0

In [ ]:
infill_transformation = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((64, 64), antialias = True)
])

infiller_back_to_array = transforms.Compose([
    
])

In [ ]:
def infilling_pipeline(image, original_shape):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    with torch.no_grad():
        img = infill_transformation(image)[None, ...]

        img = img.to(device)
        img = infiller(img)[0]
        img = transforms.Resize(original_shape, antialias = False)(img)
        img =  img.detach().cpu().permute(1,2,0).numpy()
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return img*255.

## Masked

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
nose_cascade = cv2.CascadeClassifier( cv2.data.haarcascades + 'haarcascade_mcs_nose.xml')

def getMask(img):
  # Load the image
  # img = cv2.imread(imagePath)
  # Convert the image to grayscale
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  # Detect faces
  faces = face_cascade.detectMultiScale(gray, 1.3, 5)
  # Loop through each face and detect eyes

  # Create a mask image
  mask = np.ones(img.shape[:2], np.uint8)
  
  facesCount = 0
  for (x,y,w,h) in faces:

      roi_gray = gray[y:y+h, x:x+w]
      roi_color = img[y:y+h, x:x+w]
      eyes = eye_cascade.detectMultiScale(roi_gray)
      
      # Loop through each eye and get the coordinates
      # eye_y = img.shape[0]
      cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 4)        # create bounding box for the face
      eyeCount = len(eyes)
      lowesty = 0
      for (ex,ey,ew,eh) in eyes:
          eye_x = x + ex + int(ew/2)
          eye_y = y + ey + int(eh/2)  
          lowesty = max(lowesty, eye_y + eh//4)
        #   cv2.rectangle(img, (eye_x-ew//2, eye_y-eh//2), (eye_x+ew//2, eye_y+eh//2), (0, 0, 255), 4)
      
      face_img = img[y:y+h, x:x+w]
    
      #masked_image = cv2.bitwise_and(img, img, mask = mask)
      
      # masked or not
      facesCount += 1
        
      if(eyeCount >= 1 and masked_inference_pipeline(face_img)):
          
          # cv2.imshow(face_img)
          # cv2.waitKey(0)
          # for posx in range(x, x+w):
          #   for posy in range(lowesty, min(y+h+h//12, mask.shape[0])):
          mask[lowesty: min(y+h+h//12, mask.shape[0]), x:x+w] = 0
          masked_image = cv2.bitwise_and(img, img, mask = mask)

          face_img = masked_image[y:y+h, x:x+w]
          original_shape = face_img.shape[:2]

          # infilling
          infill = infilling_pipeline(face_img, original_shape)

          img[y:y+h, x:x+w, :] = infill


      if(facesCount > 0):
        break

  #masked_image = cv2.bitwise_and(img, img, mask = mask) 
  return img


cap = cv2.VideoCapture(0)
while True:
    # Read the frame
    _, img = cap.read()
    gray = getMask(img)
    cv2.imshow('img', gray)
    k = cv2.waitKey(30) & 0xff
    if k==27:
        break



cap.release()
cv2.destroyAllWindows()

## Unmasked

In [ ]:
# import cv2
# import numpy as np
# face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
# eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
# nose_cascade = cv2.CascadeClassifier( cv2.data.haarcascades + 'haarcascade_mcs_nose.xml')

# def getMask(img):
#   # Load the image
#   # img = cv2.imread(imagePath)
#   # Convert the image to grayscale
#   gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#   # Detect faces
#   faces = face_cascade.detectMultiScale(gray, 1.3, 5)
#   # Loop through each face and detect eyes

#   # Create a mask image
#   mask = np.ones(img.shape[:2], np.uint8)
#   for (x,y,w,h) in faces:
#       roi_gray = gray[y:y+h, x:x+w]
#       roi_color = img[y:y+h, x:x+w]
#       eyes = eye_cascade.detectMultiScale(roi_gray)
      
#       # Loop through each eye and get the coordinates
#       # eye_y = img.shape[0]
#       cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 4)
#       eyeCount = len(eyes)
#       lowesty = 0
#       for (ex,ey,ew,eh) in eyes:
#           eye_x = x + ex + int(ew/2)
#           eye_y = y + ey + int(eh/2)  
#           lowesty = max(lowesty, eye_y + eh//3)
#           cv2.rectangle(img, (eye_x-ew//2, eye_y-eh//2), (eye_x+ew//2, eye_y+eh//2), (0, 0, 255), 4)
        
#       if(eyeCount >= 1 and True):
#         for posx in range(x, x+w):
#           for posy in range(lowesty, y+h):
#             mask[posy][posx] = 0
#   masked_image = cv2.bitwise_and(img, img, mask=mask)
#   # Display the image
#   return masked_image


# cap = cv2.VideoCapture(0)
# while True:
#     # Read the frame
#     _, img = cap.read()
#     gray = getMask(img)
#     cv2.imshow('img', gray)
#     k = cv2.waitKey(30) & 0xff
#     if k==27:
#         break


# cap.release()
# cv2.destroyAllWindows()